In [196]:
from sqlalchemy import create_engine
import pandas as pd
import json
import requests
from time import sleep
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
import re
import json
import folium
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import branca.colormap as cm
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm

municipality_df = pd.read_csv("../data/municipalities_3_metricized.csv")

engine = create_engine('sqlite:///../data/municipalities_3.db')

municipality_df = pd.read_sql_table('municipality', engine)
print("Total number of municipalitites: ", len(municipality_df))
print("amount of municipalities with their name in the content: ", len(municipality_df[municipality_df.apply(lambda row: row['name'].lower() in row['content'].lower(), axis=1)]))
print("amount of municipalitites with connection error: ", len(municipality_df[municipality_df["content"].str.contains("Cannot connect to host")]))
print("Total number of clean municipalities: ", len(municipality_df[municipality_df.apply(lambda row: ("Cannot connect to host" not in row['content']) and (row['name'] in row['content']), axis=1)]))
print("Amount of rows with no html content: ", len(municipality_df[municipality_df.apply(lambda row: (row["content"] == ""), axis = 1)]))
print("Amount of rows with '' as content: ", len(municipality_df[municipality_df.apply(lambda row: (row["content"] == ""), axis = 1)]))
print("Amount of rows with 'None' as content: ", len(municipality_df[municipality_df.apply(lambda row: (row["content"] == None), axis = 1)]))

municipality_df = municipality_df[municipality_df.apply(lambda row: ("Cannot connect to host" not in row['content']) and (row['name'] in row['content']), axis=1)]

Total number of municipalitites:  5087
amount of municipalities with their name in the content:  4047
amount of municipalitites with connection error:  77
Total number of clean municipalities:  3866
Amount of rows with no html content:  65
Amount of rows with '' as content:  65
Amount of rows with 'None' as content:  0


In [194]:
municipality_df

,id,name,country,email,population,pop_d,status,content,url,scripts,...,external_links_count,internal_links_count,image_count,scripts_count,styles_count,Latitude,Longitude,mediaqueries_count_processed,responsiveness,scaled_responsiveness
0,4,Delvinë,Albania,info@bashkiadelvine.gov.al,7887,NaN,"no email found, email added from teams version","<!DOCTYPE html>\n<html lang=""sq-AL"">\n<head da...",http://www.bashkiadelvine.gov.al,[],...,190.0,17.0,30.0,40.0,8.0,39.972035,20.090346,0.137880,0.137880,-0.033333
1,9,Durrës,Albania,info@durres.gov.al,175110,NaN,"no email found, email added from teams version","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",http://www.durres.gov.al,[],...,43.0,9.0,13.0,58.0,12.0,41.331057,19.456793,0.845910,0.845910,2.500000
2,38,Mirditë,Albania,info@bashkiamirdite.gov.al,22103,NaN,"no email found, email added from teams version","<!doctype html>\n<html lang=""en-US"">\n<head>\n...",http://bashkiamirdite.gov.al,[],...,45.0,3.0,12.0,33.0,14.0,41.859411,20.018134,0.497485,0.497485,1.253333
3,46,Pustec,Albania,info@bashkiapustec.gov.al,3290,NaN,"no website found, email added from teams versi...","<!DOCTYPE html>\n<html lang=""sq"">\n<head data-...",https://www.bashkiapustec.gov.al/,[],...,160.0,19.0,32.0,45.0,8.0,40.787890,20.901217,0.137880,0.137880,-0.033333
4,62,Canillo,Andorra,comucanillo@andorra.ad,NaN,NaN,no status available,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",http://www.canillo.ad,[],...,35.0,22.0,29.0,9.0,0.0,42.566831,1.597765,0.162102,0.162102,0.053333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3861,90491,Causeway Coast and Glens,UK,info@causewaycoastandglens.gov.uk,143920,NaN,no error,"<!doctype html>\n<html class=""no-js"" lang=""en""...",http://www.causewaycoastandglens.gov.uk,[],...,36.0,72.0,34.0,10.0,1.0,55.130595,-6.327746,0.190050,0.190050,0.153333
3862,90501,Clackmannanshire,UK,customerservice@clacks.gov.uk,51450,NaN,no error,"\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang=""en""...",http://www.clacks.gov.uk/,[],...,25.0,92.0,17.0,7.0,2.0,56.166395,-3.751397,0.087572,0.087572,-0.213333
3863,90508,Perth and Kinross,UK,NaN,151100,NaN,no email found,"\r\n<!DOCTYPE html>\r\n<html lang=""en"">\r\n<he...",http://www.pkc.gov.uk/,[],...,187.0,0.0,10.0,18.0,1.0,56.500000,-3.750000,0.296255,0.296255,0.533333
3864,90540,Isle of Anglesey,UK,NaN,69794,NaN,no email found,"<!DOCTYPE html>\r\n<html lang=""en-GB"">\r\n\r\n...",http://www.anglesey.gov.uk,[],...,0.0,2.0,1.0,4.0,14.0,53.278160,-4.347871,0.240358,0.240358,0.333333


In [3]:
social_media_platforms = ['facebook', 'twitter', 'instagram', 'linkedin', 'youtube', 'pinterest', 'plus.google']
service_keywords = ['form']

metrics_dict = {
        'links_count': None,
        'forms_count': None,
        'social_media_count': None,
        'mediaqueries_count': None,
        'grid_count': None,
        'iframe_count': None,
        'pdf_count': None,
        'worddoc_count': None,
        'external_links_count': None,
        'internal_links_count': None,
        'image_count': None,
        'scripts_count': None,
        'styles_count': None
    }

def process_row(index, row):
    soup = BeautifulSoup(row['content'], 'html.parser')

    # static metrics
    links = soup.find_all('a')
    forms = soup.find_all('form')
    grids = soup.find_all(['div', 'span'], style=re.compile("grid")) # find grid layout usage
    iframes = soup.find_all('iframe')
    pdfs = soup.find_all(href=re.compile(".pdf$")) # links ending with .pdf
    word_docs = soup.find_all(href=re.compile(".word$")) # links ending with .word

    external_links = sum(1 for link in links if link.get('href', '').startswith('http'))
    internal_links = len(links) - external_links
    images = soup.find_all('img')
    scripts = soup.find_all('script')
    styles = soup.find_all('style')
    
    social_media_count = sum(any(platform in link.get('href', '') for platform in social_media_platforms) for link in links)

    css_files = row['css_files']
    css_dict = json.loads(css_files)
    
    # If the dictionary is empty, return metrics as 0
    if not css_dict:
        mediaqueries_count = 0
    else:
        mediaqueries_count = sum(css_content.count('@media') for css_content in css_dict.values() if css_content)

    return index, {
        'links_count': len(links),
        'forms_count': len(forms),
        'social_media_count': social_media_count,
        'mediaqueries_count': mediaqueries_count,
        'grid_count': len(grids),
        'iframe_count': len(iframes),
        'pdf_count': len(pdfs),
        'worddoc_count': len(word_docs),
        'external_links_count': external_links,
        'internal_links_count': internal_links,
        'image_count': len(images),
        'scripts_count': len(scripts),
        'styles_count': len(styles)
    }

results = Parallel(n_jobs=-1)(delayed(process_row)(index, row) for index, row in municipality_df.iterrows())

for original_index, result in results:
    for metric, value in result.items():
        municipality_df.at[original_index, metric] = value

municipality_df.head()

   id     name  country                       email population  pop_d  \
0   4  Delvinë  Albania  info@bashkiadelvine.gov.al       7887    NaN   
1   9   Durrës  Albania          info@durres.gov.al     175110    NaN   
2  38  Mirditë  Albania  info@bashkiamirdite.gov.al      22103    NaN   
3  46   Pustec  Albania   info@bashkiapustec.gov.al       3290    NaN   
4  62  Canillo  Andorra      comucanillo@andorra.ad       None    NaN   

                                              status  \
0     no email found, email added from teams version   
1     no email found, email added from teams version   
2     no email found, email added from teams version   
3  no website found, email added from teams versi...   
4                                no status available   

                                             content  \
0  <!DOCTYPE html>\n<html lang="sq-AL">\n<head da...   
1  <!DOCTYPE html>\n<html lang="en-US" prefix="og...   
2  <!doctype html>\n<html lang="en-US">\n<head>\n...   


In [4]:
metrics_dict.keys()

dict_keys(['links_count', 'forms_count', 'social_media_count', 'mediaqueries_count', 'grid_count', 'iframe_count', 'pdf_count', 'worddoc_count', 'external_links_count', 'internal_links_count', 'image_count', 'scripts_count', 'styles_count'])

In [5]:
from tqdm import tqdm
def get_coordinates(municipality, country):
    sleep(1)
    # Format the address for the request
    address = f"{municipality}, {country}"
    
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": address,
        "format": "json"
    }
    
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # Raise an exception if the request was unsuccessful
    
    data = response.json()
    if data:
        latitude = float(data[0]["lat"])
        longitude = float(data[0]["lon"])
        return latitude, longitude
    else:
        print(f"No coordinates found for {address}")
        return None, None

# Use the function to get coordinates for each row in the dataframe
for i, row in tqdm(municipality_df.iterrows(), total=len(municipality_df)):
    municipality_df.at[i, "Latitude"], municipality_df.at[i, "Longitude"] = get_coordinates(row["name"], row["country"])

100%|██████████| 3866/3866 [1:16:28<00:00,  1.19s/it]


In [6]:
municipality_df.to_csv("../data/municipalities_3_metricized.csv", index=False)

In [7]:
pages_df = pd.read_sql_table('pages', engine)
municipality_df_sublinks = municipality_df[municipality_df['id'].isin(pages_df['municipality_id'].unique())]
municipality_df_sublinks

,id,name,country,email,population,pop_d,status,content,url,scripts,...,iframe_count,pdf_count,worddoc_count,external_links_count,internal_links_count,image_count,scripts_count,styles_count,Latitude,Longitude
10,153,Nickelsdorf,Austria,gemeinde@nickelsdorf.at,1809,NaN,no error,"<!DOCTYPE html> <html lang=""de"" dir=""ltr"" clas...",http://www.nickelsdorf.at,[],...,0.0,12.0,0.0,67.0,226.0,23.0,5.0,3.0,47.940302,17.066176
54,856,Kötschach-Mauthen,Austria,glasfaseranschluss@kelag.at,3321,NaN,no error,"<!DOCTYPE html><html lang=""de-AT"" xmlns:og=""ht...",http://www.koetschach-mauthen.at,[],...,0.0,0.0,0.0,111.0,6.0,23.0,7.0,1.0,46.661677,12.971170
81,1283,Grieskirchen,Austria,rathaus@grieskirchen.at,4984,NaN,no error,\r\n <!DOCTYPE html>\r\n\r\n\r\n <html l...,http://www.grieskirchen.at/,[],...,0.0,0.0,0.0,17.0,130.0,7.0,34.0,1.0,48.235011,13.826192
263,5599,Dobrá Voda u Českých Budějovic,Czech_Republic,dobra.voda.cb@obce.cz,2675,1729.057079,no status available,"<!DOCTYPE html>\n<html lang=""cs"">\n<head>\n\t<...",http://www.dobravodaucb.cz/,[],...,0.0,2.0,0.0,10.0,92.0,9.0,22.0,0.0,48.968092,14.524649
299,5916,Halámky,Czech_Republic,halamky@quick.cz,194,28.136335,no status available,"<!DOCTYPE html>\n<html lang=""cs"" data-lang-sys...",http://www.halamky.info/,[],...,0.0,0.0,0.0,5.0,40.0,5.0,12.0,0.0,48.853137,14.915125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3599,84542,Fuenllana,Spain,NaN,214,NaN,no email found,"<!DOCTYPE html><html lang=""es"" prefix=""og: htt...",https://fuenllana.com/,[],...,1.0,0.0,0.0,13.0,2.0,22.0,22.0,10.0,38.755953,-2.958387
3635,85541,Sant Esteve Sesrovires,Spain,NaN,7771,NaN,no email found,"<!DOCTYPE HTML>\n<html lang=""ca"" xmlns=""http:/...",http://www.sesrovires.cat/,[],...,2.0,0.0,0.0,250.0,57.0,32.0,36.0,1.0,41.493243,1.874735
3645,85708,Fontcoberta,Spain,NaN,1468,NaN,no email found,"<!doctype html>\n<html lang=""ca"">\n\n<head>\n ...",http://www.fontcoberta.cat/,[],...,0.0,0.0,0.0,58.0,7.0,8.0,28.0,7.0,42.142538,2.789782
3697,86858,Cheles,Spain,NaN,1172,NaN,no email found,\n\n\n \n \n\n\t\n<!DOCTYPE ht...,http://www.cheles.es,[],...,0.0,0.0,0.0,23.0,84.0,36.0,17.0,0.0,38.512580,-7.281548


In [12]:
for i, row in tqdm(municipality_df_sublinks.iterrows(), total = len(municipality_df_sublinks)):
    tmp = pages_df[pages_df["municipality_id"] == row["id"]]
    municipality_df_sublinks.at[i, "subpages_forms_count"] = 0
    municipality_df_sublinks.at[i, "subpages_iframes_count"] = 0
    municipality_df_sublinks.at[i, "subpages_pdfs_count"] = 0
    municipality_df_sublinks.at[i, "subpages_word_docs_count"] = 0
    municipality_df_sublinks.at[i, "subpages_images_count"] = 0

    for j, row_subpage in tmp.iterrows():
        soup = BeautifulSoup(row_subpage['content'], 'html.parser')

        # static metrics
        forms = soup.find_all('form')
        iframes = soup.find_all('iframe')
        pdfs = soup.find_all(href=re.compile(".pdf$")) # links ending with .pdf
        word_docs = soup.find_all(href=re.compile(".word$")) # links ending with .word
        images = soup.find_all('img')

        municipality_df_sublinks.at[i, "subpages_forms_count"] += len(forms)
        municipality_df_sublinks.at[i, "subpages_iframes_count"] += len(iframes)
        municipality_df_sublinks.at[i, "subpages_pdfs_count"] += len(pdfs)
        municipality_df_sublinks.at[i, "subpages_word_docs_count"] += len(word_docs)
        municipality_df_sublinks.at[i, "subpages_images_count"] += len(images)

  0%|          | 0/84 [00:00<?, ?it/s]/tmp/ipykernel_94597/73981045.py:10: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(row_subpage['content'], 'html.parser')
  1%|          | 1/84 [00:06<08:25,  6.10s/it]/tmp/ipykernel_94597/73981045.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row_subpage['content'], 'html.parser')
  7%|▋         | 6/84 [00:16<02:26,  1.87s/it]/home/bruno/anaconda3/envs/muni-crawler/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know

In [11]:
municipality_df_sublinks.to_csv("../data/municipality_df_sublinks_metricized.csv", index=False)